# Morse En/Decoder
#### By Ethan Sim

In theory, encoding and decoding Morse is quite simple. The challenge for me will be to implement this whole thing in a systematic manner, and hopefully to arrive at a simple user interface.

Generally, our workflow will be as follows:
- Create a dictionary to translate alphabets and special characters into Morse and vice versa
- Create a testing suite for each en/decoder to iron out any mistakes
- Wrap this in a user interface to allow for input of plaintext and ciphertext for encoder and decoder respectively

## Dictionary Creation

First, we create the dictionary **morse_dict**, which will contain all character-to-Morse links. **morse_dict** shall have the following properties:
- Allow all letters to be translated into Morse
- Allow all numbers to be translated into Morse
- Permit separation of individual letters and words
- Allow certain special characters (e.g. punctuation) to be translated into Morse (I will ignore the non-latin and prosign tables here)

<img src="alphabet.svg">

**Figure 1: The basis of our Morse translation. Source: https://www.cryptomuseum.com/radio/morse/**

In [92]:
morse_dict = {" ":"/ ", "A":".- ", "B":"-... ", "C":"-.-. ", "D":"-.. ", "E":". ", "F":"..-. ", "G":"--. ", 
              "H":".... ", "I": ".. ", "J":".--- ", "K":"-.- ", "L":".-.. ", "M":"-- ", "N":"-. ", "O":"--- ",
              "P":".--. ", "Q":"--.- ", "R":".-. ", "S":"... ", "T":"- ", "U":"..- ", "V":"...- ", "W":".-- ",
              "X":"-..- ", "Y":"-.-- ", "Z":"--.. ", "1":".---- ", "2":"..--- ", "3":"...-- ", "4":"....- ",
              "5":"..... ", "6":"-.... ", "7":"--... ", "8":"---.. ", "9":"----. ", "0":"----- ", ".":".-.-.- ",
              ",":"--..-- ", "?":"..--.. ", "'":".----. ", "!":"-.-.-- ", "/":"-..-. ", "(":"-.--.", ")":"-.--.- ",
              "&":".-... ", ":":"---... ", ";":"-.-.-. ", "=":"-...- ", "+":".-.-. ", "-":"-....- ", "_":"..--.- ",
              '"':".-..-. ", "$":"...-..- ", "@":".--.-. "}

#Creating these lists to save memory when looking up values
plaintext_characters = list(morse_dict.keys())
ciphertext_characters = list(morse_dict.values())

Our dictionary has the following properties (and shortcomings):
- I have used spaces to separate individual characters, and slashes to separate individual words. This will enable my decoder to operate reliably later on.
- Unfortunately there is no way to preserve the capital/small letter information in Morse - traditionally, plaintext was all uppercase.
- I've used different colon marks to capture the two different quotation styles; hopefully this works.

## Encoding

With the dictionary settled, we now turn to the encoding - turning user input into Morse. We have to coerce our user input to uppercase - otherwise our dictionary won't work!

In [98]:
import re

#To refactor our code, let's use a helper function
def whitespace_sorter(sentence):
    """
    Takes as input a string sentence and removes leading and trailing whitespace, also coerces all multiple whitespaces into a single whitespace character.
    """
    sentence_copy = str(sentence)
    sentence_copy = sentence_copy.strip() #Remove leading and trailing whitespace (/s)
    sentence_copy = re.sub(" +", " ", sentence_copy) #Coerces all multiple /s characters into a single /s
    #It identifies a /s followed by any nonzero number of /s and replaces this with a single /s 
    return sentence_copy

def encode_morse(plaintext):
    """
    Takes as input a string plaintext and returns its Morse equivalent in string form.
    """
    if not isinstance(plaintext, str):
        return "Plaintext is not a string!"
    # Having confirmed it's a string, we convert it to uppercase - this will leave numbers and special characters untouched
    if not plaintext.isupper():
        plaintext_copy = plaintext.upper() #We don't want to mutate the input
    else:
        plaintext_copy = str(plaintext)
    plaintext_copy = whitespace_sorter(plaintext_copy)        
    ciphertext = "" #This also has the effect of returning an empty string if an empty string is the input
    #We then do the actual translation by simply looking up the dictionary value
    for character in plaintext_copy:
        if character not in plaintext_characters:
            return "You can't encode the following character: " + character
        ciphertext += morse_dict[character]
    return ciphertext[:-1] #Remove trailing /s

Now that we've created our encoder, let's build our testing suite. We will check our output against that provided by https://morsecode.world/international/translator.html, which uses the same space and character delimiters. Note that this website cannot translate certain characters, such as $.

In [167]:
def test_encoder():
    """
    This will run our Morse encoder against a selection of varying inputs, and compare the outputs to a model answer.
    """
    #Check edge cases first
    assert encode_morse(123) == "Plaintext is not a string!", "Test 1 failed, input integer 123"
    assert encode_morse("") == "", "Test 2 failed, input ''"
    assert encode_morse("^") == "You can't encode the following character: ^", "Test 3 failed, input '^'"
    assert encode_morse("   e       e   ") == ". / .", "Test 4 failed, input '   e       e   '"
    assert encode_morse("AbCd") == ".- -... -.-. -..", "Test 5 failed, input 'AbCd'"
    
    #Now we run possible plaintexts and check their corresponding ciphertexts
    assert encode_morse("the quick brown fox jumps over the lazy dog") == "- .... . / --.- ..- .. -.-. -.- / -... .-. --- .-- -. / ..-. --- -..- / .--- ..- -- .--. ... / --- ...- . .-. / - .... . / .-.. .- --.. -.-- / -.. --- --.", "Test 6 failed, input 'the quick brown fox jumps over the lazy dog'"
    assert encode_morse("H1er0ph@nT + '") == ".... .---- . .-. ----- .--. .... .--.-. -. - / .-.-. / .----.", "Test 7 failed, input 'H1er0ph@nT + ''"
    assert encode_morse('"' + "'") == ".-..-. .----.", "Test 8 failed, input ''(double apostrophe)' + '(single apostrophe)'"
    
    #Check that input not mutated
    test_plaintext_9 = "test"
    encode_morse(test_plaintext_9)
    assert test_plaintext_9 == "test", "Test 9 failed, input 'test' mutated"
    
    #If all tests passed
    print ("Congratulations! 9/9 tests passed!")

test_encoder()

Congratulations! 9/9 tests passed!


Having validated our encoding function, we can now present it to the user.

In [184]:
#Press enter once input done - else the kernel will freeze.
print("Enter your plaintext:")
en_plaintext = str(input())
print ("Your Morse ciphertext is:")
en_ciphertext = encode_morse(en_plaintext)
print (en_ciphertext)

Enter your plaintext:


Your Morse ciphertext is:



## Decoding

Now, we turn to decoding. We will specify rules for user input to ensure our dictionary works, such as the use of spaces to delimit letters and slashes to separate words. 
- In line with our model answer, we need to ignore leading, successive, and trailing slashes. We can, however, cheat a bit by simply stripping the decoded plaintext once the heavy lifting is done by the dictionary. 
- A weakness of the model website is that it parses a single slash as plaintext rather than ciphertext, although this makes sense because only the output of the former has meaning. 
- As mentioned previously, our plaintext output will be in all capitals.

In [152]:
def decode_morse(ciphertext):
    """
    Takes as input the Morse string ciphertext and returns a plaintext string.
    """
    if not isinstance(ciphertext, str):
        return "Ciphertext is not a string!"
    ciphertext_copy = str(ciphertext)
    if len(ciphertext) == 0: #Accounts for empty string
        return ""
    if ciphertext_copy[-1] != " ":
        ciphertext_copy += " " #Accounts for user variation in final trailing whitespace - we need this final whitespace for the dictionary to work
        #This also has the effect of returning nonsense characters we can't decode later on
    plaintext = "" #Empty string solution
    morse_char = "" #This variable will hold each letter/character's Morse code
    for character in ciphertext_copy:
        if character == " ": #Spaces are letter delimiters
            morse_char += character
            if morse_char in ciphertext_characters:
                plaintext += plaintext_characters[ciphertext_characters.index(morse_char)]
                morse_char = "" #Reset the holding variable
            else:
                return "Your decoded message thus far is: " + whitespace_sorter(plaintext) + "\nI can't decode the following character: " + morse_char
                #The nature of this return statement allows tests via assertion, but will also respond to print statements accordingly.
        else:
            morse_char += character #If it's not a letter delimiter, continue building the letter/character Morse code
    plaintext = whitespace_sorter(plaintext)
    return plaintext

Now that we've created our decoder, let's build our testing suite. We check our output against that of the website listed above. I could check it against **encode_morse()**, but let's be impartial and use an external reference.

In [173]:
def test_decoder():
    """
     This will run our Morse decoder against a selection of varying inputs, and compare the outputs to a model answer.
    """
    #Check edge cases first
    assert decode_morse(123) == "Ciphertext is not a string!", "Test 1 failed, input integer 123"
    assert decode_morse("") == "", "Test 2 failed, input ''"
    assert decode_morse("string") == "Your decoded message thus far is: \nI can't decode the following character: string ", "Test 3 failed, input 'string'"
    assert decode_morse(".- ..- / .- . .--.-.-.-.-.-.-.-.-.-.") == "Your decoded message thus far is: AU AE\nI can't decode the following character: .--.-.-.-.-.-.-.-.-.-. ", "Test 4 failed, input '.- ..- / .- . .--.-.-.-.-.-.-.-.-.-.'" 
    assert decode_morse("/") == "", "Test 5 failed, input '/'" #My function parses the slash as ciphertext, but whitespace_sorter discards it as meaningless noise.
    #This is fair because both encoder and decoder ignore spaces presented by themselves, in plaintext and ciphertext respectively.
    
    #Now we run possible ciphertexts and check their corresponding plaintexts:
    assert decode_morse("- .... . / --.- ..- .. -.-. -.- / -... .-. --- .-- -. / ..-. --- -..- / .--- ..- -- .--. ... / --- ...- . .-. / - .... . / .-.. .- --.. -.-- / -.. --- --.") == "THE QUICK BROWN FOX JUMPS OVER THE LAZY DOG", "Test 6 failed, input '- .... . / --.- ..- .. -.-. -.- / -... .-. --- .-- -. / ..-. --- -..- / .--- ..- -- .--. ... / --- ...- . .-. / - .... . / .-.. .- --.. -.-- / -.. --- --.'"
    assert decode_morse(".... .---- . .-. ----- .--. .... .--.-. -. - / .-.-. / .----.") == "H1ER0PH@NT + '", "Test 7 failed, input '.... .---- . .-. ----- .--. .... .--.-. -. - / .-.-. / .----."
    assert decode_morse(".-..-. .----.") == '"' + "'", "Test 8 failed, input '.-..-. .----.'"
    
    #Check that input not mutated
    test_ciphertext_9 = "- . ... -"
    encode_morse(test_ciphertext_9)
    assert test_ciphertext_9 == "- . ... -", "Test 9 failed, input '- . ... -' mutated" 
    
    #If all tests passed
    print ("Congratulations! 9/9 tests passed!")

test_decoder()

Congratulations! 9/9 tests passed!


Having validated our decoding function, we can now present it to the user.

In [185]:
print("Enter your Morse ciphertext:")
de_ciphertext = str(input())
print ("Your decoded plaintext is:")
de_plaintext = decode_morse(de_ciphertext)
print (de_plaintext)

Enter your Morse ciphertext:


Your decoded plaintext is:



## Wrapping Up

Now that we've settled both encoder and decoder, let's try to present both in a simple user interface.

In [188]:
def morse_interface(de_en_flag):
    """
    A simple wrapper in Jupyter Notebook to enable encoding to and decoding from Morse.
    """
    if de_en_flag == "decode":
        print("Enter your Morse ciphertext:")
        user_ciphertext = str(input())
        print ("Your decoded plaintext is:")
        print (decode_morse(user_ciphertext))
    elif de_en_flag == "encode":
        print("Enter your plaintext:")
        user_plaintext = str(input())
        print ("Your Morse ciphertext is:")
        print (encode_morse(user_plaintext))
    else:
        print ("Invalid input!")    

And now that we've created this very basic interface, we can now present the following to the user - comment out the line below to try it!

In [189]:
print("Would you like to encode or decode? Type 'encode' to encode plaintext into Morse, and 'decode' to decode a Morse ciphertext.")
#morse_interface(str(input()))

Would you like to encode or decode? Type 'encode' to encode plaintext into Morse, and 'decode' to decode a Morse ciphertext.
